In [ ]:
import numpy as np
from math import exp, sqrt
import time
import numba

@numba.jit(nopython=True)
def payoff_asian_call(paths: np.ndarray, K: float, dt: float, T: float) -> np.ndarray:
    """
     Calculates the payoff for an Asian call option

    :param paths: Numpy array of simulated asset prices
    :param K: Strike price
    :param dt: Time step size
    :param T: Total time period
    :return: Numpy array of payoffs for the simulated path
    """
    num_paths = paths.shape[0]
    payoffs = np.zeros(num_paths)
    for i in range(num_paths):
        trapezoidal_sum = 0.0
        # Approximate the integral by using adjacent points
        for j in range(paths.shape[1] - 1):
            trapezoidal_sum += 0.5 * (paths[i, j] + paths[i, j+1])

        # Get average
        integral = trapezoidal_sum * dt
        S_bar = integral / T

        payoffs[i] = max(S_bar - K, 0.0)
    return payoffs

@numba.jit(nopython=True)
def euler_path(S0: float, dW: np.ndarray, dt: float, r: float, sigma: float) -> np.ndarray:
    """
    Simulate the asset price for GBM using the Euler-Maruyama scheme

    :param S0: Initial asset price
    :param dW: 2d array of Wiener increments
    :param dt: Time step size
    :param r: Risk-free rate
    :param sigma: Volatility of the asset
    :return: Numpy array of complete simulated price paths
    """
    num_paths, num_steps = dW.shape

    #Initialise initial point and subsequent steps
    paths = np.zeros((num_paths, num_steps + 1))
    paths[:, 0] = S0
    for j in range(num_steps):
        s = paths[:, j]
        paths[:, j+1] = s + r * s * dt + sigma * s * dW[:, j]
    return np.maximum(paths, 1e-12)

@numba.jit(nopython=True)
def gbm_exact_path(S0:float, dW:float, dt:float, r:float, sigma:float):
    """
    Simulate asset price using exact solution for GBM

    :param S0: Initial stock price
    :param dW: 2d array of Wiener increments
    :param dt: Size of each time step
    :param r: Risk free rate
    :param sigma: Volatility

    :return: Array of simulated asset price paths
    """
    N, n = dW.shape
    paths = np.zeros((N, n+1))
    paths[:, 0] = S0

    # Drift term for the exact GBM path simulation
    drift = (r - 0.5*sigma*sigma) * dt
    for j in range(n):
        paths[:, j+1] = paths[:, j] * np.exp(drift + sigma * dW[:, j])
    return paths

@numba.jit(nopython=True)
def mlmc_sample_level_asian(path_simulator, S0: float, K: float, r:float, sigma: float, T: float, l:int, N_l: int, rng, M=4):

    """
    Generates samples for a single level of the MLMC estimator

    :param path_simulator:  Numerical scheme to use
    :param S0: Initial stock price
    :param K: Strike price
    :param r: Risk-free rate
    :param sigma: Volatility
    :param T: Time to maturity
    :param l: Current MLMC level
    :param N_l: Number of samples to generate for this level
    :param rng: Random number generator
    :param M: Refinement factor

    :return: Tuple containing the samples and computational cost
    """
    n_fine = M**l
    dt_f = T / n_fine

    # Cost if n fine for 0 but n fine + n coarse for other levels as 2 paths must be simulated
    cost = n_fine if l == 0 else (n_fine + n_fine // M)

    # Generate the Wiener process increments for finest path
    dW_f = rng.normal(0.0, sqrt(dt_f), size=(N_l, n_fine))

    if l == 0:
        paths_f = path_simulator(S0, dW_f, dt_f, r, sigma)
        Y = exp(-r * T) * payoff_asian_call(paths_f, K, dt_f, T)
    else:
        # Generate coarse path by summing fine increments
        dW_c = dW_f.reshape(N_l, n_fine // M, M).sum(axis=2)
        dt_c = T / (n_fine // M)
        paths_f = path_simulator(S0, dW_f, dt_f, r, sigma)
        paths_c = path_simulator(S0, dW_c, dt_c, r, sigma)

        payoff_f = payoff_asian_call(paths_f, K, dt_f, T)
        payoff_c = payoff_asian_call(paths_c, K, dt_c, T)
        Y = exp(-r * T) * (payoff_f - payoff_c)
    return Y, cost

def mlmc_adaptive_asian(scheme_name, path_simulator, S0:float, K:float, r:float, sigma:float, T:float, eps:float=0.01, seed:float=651028):
    """
    Implements MLMC for Asian Option

    :param scheme_name: Name of scheme
    :param path_simulator: Numerical function to use
    :param S0: Initial stock price
    :param K: Strike price
    :param r: Risk-free rate
    :param sigma: Volatility
    :param T: Time to maturity
    :param eps: RMSE tolerance
    :param seed: Random seed
    """
    print(f"\n=== Adaptive MLMC for Asian Option ({scheme_name} Scheme) ===")
    start_time = time.time()
    rng = np.random.default_rng(seed)
    L, L_max, N0, M = 2, 10, 2000, 4
    means = np.zeros(L_max + 1)
    vars = np.zeros(L_max + 1)
    Nl = np.zeros(L_max + 1, dtype=int)
    costs = np.zeros(L_max + 1)

    # Initial sampling to estimate moments and costs for the first levels
    for l in range(L + 1):
        Nl[l] = max(100, N0 // (M**l))
        Y, cost = mlmc_sample_level_asian(path_simulator, S0, K, r, sigma, T, l, Nl[l], rng, M)
        means[l], vars[l], costs[l] = np.mean(Y), np.var(Y, ddof=1), cost

    # Adaptive loop for optimal allocation and convergence
    for _ in range(15):

        # The variance target is typically half of the total MSE target
        total_var = sum(vars[l] / Nl[l] for l in range(L + 1) if Nl[l] > 0)
        if total_var < eps**2 / 2:
            break

        # Bias is the approximation if mean highest diff terms
        if L < L_max and abs(means[L]) > eps / sqrt(2):
            L += 1
            Nl[L] = max(100, N0 // (M**L))
            Y, cost = mlmc_sample_level_asian(path_simulator, S0, K, r, sigma, T, L, Nl[L], rng, M)
            means[L], vars[L], costs[L] = np.mean(Y), np.var(Y, ddof=1), cost


        sqrt_v_div_c = np.sqrt(vars[:L+1] / (costs[:L+1] + 1e-10))
        total_sqrt_vc = np.sum(np.sqrt(vars[:L+1] * costs[:L+1]))
        N_opt = np.ceil((2.0 / (eps**2)) * total_sqrt_vc * sqrt_v_div_c)

        # Generate more sample to meet new allocation
        for l in range(L + 1):
            extra = max(0, int(N_opt[l] - Nl[l]))
            if extra > 0:
                Y_ex, _ = mlmc_sample_level_asian(path_simulator, S0, K, r, sigma, T, l, extra, rng, M)
                total_N = Nl[l] + extra
                new_mean = (Nl[l] * means[l] + np.sum(Y_ex)) / total_N

                # Use combined variance to update variance when adding new points
                if Nl[l] > 1 and extra > 1:
                    vars[l] = ((Nl[l] - 1) * vars[l] + (extra - 1) * np.var(Y_ex, ddof=1) +
                               Nl[l] * (means[l] - new_mean)**2 +
                               extra * (np.mean(Y_ex) - new_mean)**2) / (total_N - 1)
                means[l] = new_mean
                Nl[l] = total_N

    price = sum(means[:L+1])
    se = sqrt(sum(vars[l] / Nl[l] for l in range(L + 1) if Nl[l] > 0))
    total_cost = sum(Nl[:L+1] * costs[:L+1])

    print(f"Final Price: {price:.6f} ± {se:.6f}")
    print(f"Total levels: {L}, Total cost: {total_cost:.0f}")
    print(f"Elapsed time: {time.time() - start_time:.2f} seconds")

def get_asian_reference_price(path_simulator, S0:float, K:float, r:float, sigma:float, T:float, n_steps:int=1024, n_paths:int=200000, seed:int=232317):
    """
    Calculates a reference price for an Asian option

    :param path_simulator: Numerical scheme to use
    :param S0: Initial stock price
    :param K: Strike price
    :param r: Risk-free rate
    :param sigma: Volatility
    :param T: Time to maturity
    :param n_steps: number of time steps for the simulation
    :param n_paths:number of paths for the simulation
    :param seed: Random seed

    :return: Discounted MC estimate
    """
    print("Calculating reference price for Asian option")
    rng = np.random.default_rng(seed)
    dt = T / n_steps
    dW = rng.normal(0.0, sqrt(dt), size=(n_paths, n_steps))
    paths = path_simulator(S0, dW, dt, r, sigma)
    payoffs = payoff_asian_call(paths, K, dt, T)
    price = exp(-r * T) * np.mean(payoffs)
    print(f"Reference Price: {price:.6f}")
    return price

def weak_order_convergence_asian(scheme_name, path_simulator, ref_price:float, S0:float, K:float, r:float, sigma:float, T:float, Npaths:int =100000, seed:int =65654):
    """

    :param scheme_name: Name of scheme
    :param path_simulator: Numerical function to use
    :param ref_price: Reference price
    :param S0: Initial stock price
    :param K: Strike price
    :param r: Risk-free rate
    :param sigma: Volatility
    :param T: Time to maturity
    :param Npaths: number of paths for the simulation
    :param seed: Random seed

    :return: None but prints the weak order
    """
    print(f"\n=== Weak Order Convergence for Asian Option ({scheme_name} Scheme) ===")
    rng = np.random.default_rng(seed)
    results = []

    # Use geometric progresion for loop
    steps_list = [4, 8, 16, 32, 64, 128]

    for n in steps_list:
        dt = T / n
        dW = rng.normal(0.0, sqrt(dt), size=(Npaths, n))
        paths = path_simulator(S0, dW, dt, r, sigma)
        P_approx = exp(-r * T) * payoff_asian_call(paths, K, dt, T)
        bias = np.mean(P_approx) - ref_price
        results.append((dt, abs(bias)))
        print(f"Steps: {n:3d}, dt: {dt:.4f}, bias: {abs(bias):.3e}")

    dts = np.array([r[0] for r in results if r[1] > 0])
    biases = np.array([r[1] for r in results if r[1] > 0])

    # Need at least two non-zero data points to fit a line.
    if len(dts) < 2: return

    log_dts, log_biases = np.log(dts), np.log(biases)
    slope, _ = np.polyfit(log_dts, log_biases, 1)
    print(f"Estimated weak order for {scheme_name}: {slope:.3f}")

if __name__ == "__main__":
    params = {'S0': 1.0, 'K': 1.0, 'r': 0.05, 'sigma': 0.20, 'T': 1.0}
    ref_price = get_asian_reference_price(gbm_exact_path, **params)

    weak_order_convergence_asian("Euler", euler_path, ref_price, **params)
    mlmc_adaptive_asian("Euler", euler_path, **params, eps=0.001)